In [1]:
# import the necessary packages
from pyimagesearch import config
from imutils import paths
import random
import shutil
import os
# grab the paths to all input images in the original input directory 
# and shuffle the
imagePaths = list(paths.list_images(config.ORIG_INPUT_DATASET))
random.seed(42)
random.shuffle(imagePaths)
# compute the training and testing split
i = int(len(imagePaths) * config.TRAIN_SPLIT)
trainPaths = imagePaths[:i]
testPaths = imagePaths[i:]
# we'll be using part of the training data for validation
i = int(len(trainPaths) * config.VAL_SPLIT)
valPaths = trainPaths[:i]
trainPaths = trainPaths[i:]
# define the datasets that we'll be building
datasets = [
    ("training", trainPaths, config.TRAIN_PATH),
    ("validation", valPaths, config.VAL_PATH),
    ("testing", testPaths, config.TEST_PATH)
]

In [ ]:
# loop over the datasets
for (dType, imagePaths, baseOutput) in datasets:
    # show which data split we are creating
    print("[INFO] building '{}' split".format(dType))
    # if the output base output directory does not exist, create it
    if not os.path.exists(baseOutput):
        print("[INFO] 'creating {}' directory".format(baseOutput))
        os.makedirs(baseOutput)
    # loop over the input image paths
    for inputPath in imagePaths:
        # extract the filename of the input image and extract the
        # class label ("0" for "nagative" and "1" for "positive")
        filename = inputPath.split(os.path.sep)[-1]
        label = filename[-5:-4]
        # build the path to the label directory
        labelPath = os.path.sep.join([baseOutput, label])
        # if the label output directory does not exist, create it
        if not os.path.exists(labelPath):
            print("[INFO] 'creating {}' directory".format(labelPath))
            os.makedirs(labelPath)
        # construct the path to the destination image and then copy
        # the image itself
        p = os.path.sep.join([labelPath, filename])
        shutil.copy2(inputPath, p)

[INFO] building 'training' split
[INFO] 'creating data/split/\training' directory
[INFO] 'creating data/split/\training\0' directory
[INFO] 'creating data/split/\training\1' directory


- 이미지 압축이 아직 덜풀렸음

## CancerNet 설정

In [ ]:
# 필요 패키지
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

In [ ]:
class CancerNet:
    @staticmethod # 클래스를 부르면 자동으로 실행 되라고 
    def build(width, height, depth, classes):
        # initalize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
            
        # Conv => RELU => POOL
        model.add(SeparableConv2D(32, (3, 3), padding = "same", input_shape = inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))
        model.add(MaxPooling2D(pool_size = (2, 2)))
        model.add(Dropout(0.25))
        # (CONV => RELU => POOL) * 2
        model.add(SeparableConv2D(64, (3, 3), padding = "same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))
        model.add(SeparableConv2D(64, (3, 3), padding = "smae"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))
        model.add(MaxPooling2D(pool_size = (2, 2)))
        model.add(Dropout(0.25))
        # (CONV => RELU => POOL) * 3
        model.add(SeparableConv2D(128, (3, 3), padding = "same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))
        model.add(SeparableConv2D(128, (3, 3), padding = "smae"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))   
        model.add(SeparableConv2D(128, (3, 3), padding = "smae"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))  
        model.add(MaxPooling2D(pool_size = (2, 2)))
        model.add(Dropout(0.25))       
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(256))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        # return the constucted network architecture
        return model

## Training Script

- Create a new file named train_model.py, and insert the following

In [ ]:
# Set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

In [ ]:
# 필요 패키지 설정
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from pyimagesearch.cancernet import CancerNet
from pyimagesearch import config
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

# construct the argement parser and parse the arguments
parser = argparse.ArgumentParser()
parser.add_argument("-p", "--plot", type=str, default="plot.png",
                   help="path to output loss/accuracy plot")
args = vars(parser.parse_args(args=[]))

## Training Script - Parameters

In [ ]:
# initialize our number of epochs, initial leraning rate, and batch size
NUM_EPOCHS = 15 # 더 크게 돌려야하는데 내 컴터 성능이 구져서 쩔수임
INIT_LR = 1e-2
BS = 32

# determine the total number of image paths in training, validation, and testing directories
trainPaths = list(paths.list_images(config.TRAIN_PATH))
totalTrain = len(trainPaths)
totalVal = len(list(paths.list_images(config.VAL_PATH)))
totalTest = len(list(paths.list_images(config.TEST_PATH)))

# calculate the totla number of training images in each class and
# initialize a dictionary to store the class weights
trainLabels = [int(p.split(os.path.sep)[-2]) for p in trainPaths]
trainLabels = to_categorical(trainLabels)
classTotals = trainLabels.sum(axis = 0)
classWeight = dict()

# loop over all classes and calculate the class weight
for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]

- 이거 데이터 없어서 그럼 데이터 완료하고 다시 해봐야함

In [ ]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
    rescale = 1 / 255.0,
    rotation_range = 20,
    zoom_range = 0.05,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.05,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = "nearest")

# initialize the validation (and testing) data augmentation object
valAug = ImageDataGenerator(rescale = 1 / 255.0)

# initialize the training generator
trainGen = trainAug.flow_from_directory(
    config.TRAIN_PATH,
    class_mode = "categorical",
    target_size = (48, 48),
    color_mode = "rgb",
    shuffle = True,
    batch_size = BS)

# initialize the validation generator
valGen = valAug.flow_from_directory(
    config.VAL_PATH,
    class_mode = "categorical",
    target_size = (48, 48),
    color_mode = "rgb",
    shuffle = False,
    batch_size = BS)

# initialize the validation generator
testGen = valAug.flow_from_directory(
    config.TEST_PATH,
    class_mode = "categorical",
    target_size = (48, 48),
    color_mode = "rgb",
    shuffle = False,
    batch_size = BS)

## 모델 초기화 및 컴파일

In [ ]:
# initialize our CancerNet model and compile it
model = CancerNet.build(width=48, height=48, depth=3, classes = 2)
opt = Adagrad(learning_rate = INIT_LR, decay = INIT_LR/NUM_EPOCHS)
model.compile(loss = "binary_crossentropy", optimizer = opt,
             metrics = ["accuracy"])

# fit the model
H = model.fit(
    x = trainGen,
    steps_per_epochs = totalTrain // BS,
    validation_data = valGen,
    validation_steps = totalVal // BS,
    class_weight = classWeight,
    epochs = NUM_EPOCHS)

## 모델평가

In [ ]:
# reset the testing generator and then use our trained model to
# make predictions on the data
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict(x = testGen, steps = (totalTest // BS) + 1)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdsx, axis = 1)
# show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs,
                           target_names = testGen.class_indices.keys()))

# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity
cm = confusion_matrix(testGen.classes, predIdxs)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])

# show the confusion matrix, accuracy, sensitivity, and specificity
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

### 시각화

In [ ]:
# plot the training loss and accuracy
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label = "train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label = "val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label = "train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label = "val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

- 데이터 파일 이름 수정해야함